In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
from pathlib import Path
import re

rate_re = re.compile(r"(\(([\d.]+)%\))")
n_match = 1000
alpha = 0.05
z = stats.norm.ppf(1 - alpha / 2)
data = []

for file in Path("results/search_c").glob("*.txt"):
    C = float(file.stem)
    with file.open() as f:
        lines = f.readlines()
    # get the non-lose rate
    p = float(rate_re.search(lines[-1]).group(2)) / 100
    # get the confidence interval
    ci = z * np.sqrt(p * (1 - p) / n_match)
    data.append((C, p, ci))

df = pd.DataFrame(data, columns=["C", "p", "ci"])
df = df.sort_values("C")
df

In [ ]:
import matplotlib.pyplot as plt

X = np.arange(len(df))
plt.errorbar(X, df["p"], yerr=df["ci"], fmt="o-", capsize=3, ecolor="gray", label="Non-lose rate of UCT")
plt.xticks(X, labels=df["C"])
plt.xlabel("$C$")
plt.ylabel("$p$")
plt.title("Alpha-beta vs UCT")
plt.legend()
plt.tight_layout()
plt.savefig("results/search_c.svg")
plt.show()

In [13]:
import pandas as pd
from pathlib import Path

groups = [
    "A vs A",
    "A vs B", "B vs A",
    "A vs C", "C vs A",
    "A vs D", "D vs A",
    "A vs E", "E vs A",
]
results = []

for file in Path("results/gobang").glob("*.txt"):
    seed = int(file.stem)
    group = groups[seed // 100]
    with file.open() as f:
        lines = f.readlines()
    results.append((group, seed, lines[-1].strip()[:8]))

df = pd.DataFrame(results, columns=["group", "seed", "result"])
df.groupby(["group", "result"]).size().unstack().fillna(0).astype(int)

result,Draw,Player 1,Player 2
group,,,
A vs A,3,7,0
A vs B,3,6,1
A vs C,4,5,1
A vs D,5,4,1
B vs A,4,3,3
C vs A,8,2,0
D vs A,6,4,0
